# 并查集简介

## 并查集定义

> 并查集(Union Find)是一种用于处理不交集(Disjoint Sets)并以及查询问题的树状结构。
> 不交集指的是一系列没有重复元素的集合。
> 
> 并查集主要支持两种操作：
> - 合并(Union)：将两个集合合并为一个集合。
> - 查找(Find)：确定某个元素属于哪个集合。通常是返回集合内的一个<font color='red'>代表元素</font>。


可以定义一下「并查集」结构所支持的操作接口：
- **合并**`union(x, y)`：将集合x和集合y合并为一个集合。
- **查找**`find(x)`：查找元素x属于哪一个集合。
- **查找**`is_connected(x, y)`：查询元素x和元素y是否属于同一个集合。

## 快速合并：基于森林实现并查集

在使用「快速合并」思路实现并查集时，我们可以使用「一个森林（若干棵树）」来存储所有集合。每一棵树代表一个集合，树上的每个节点都是一个元素，树根节点为这个集合的代表元素。
> 注意：与普通的树形结构（父节点指向子节点）不同的是，基于森林实现的并查集中，树中的子节点是指向父节点的。

用数组$fa$记录森林。用$fa[x]$记录$x$的父节点的集合编号，即元素$x$指向父节点$fa[x]$。
当初始化时，$fa[x]$赋值为下标索引$x$，进行查找操作时，只要查看两个元素的根节点是否一样即可判断两个元素是否属于一个集合。进行合并操作时，只需要将两个元素的树根节点相连接（`fa[root1] = root2`）即可.

**总结一下，我们可以对数组$fa$进行以下操作来实现并查集：**

- 当初始化时：将每个元素的集合编号初始化为数组$fa$的下标索引。所有元素的根节点的集合编号不一样，代表着每个元素单独属于一个集合。
- 合并操作时：需要将两个集合的树根节点相连接。即令其中一个集合的树根节点指向另一个集合的树根节点（`fa[root1] = root2`），这样合并后当前集合中的所有元素的树根节点均为同一个。
- 查找操作时：分别从两个元素开始，通过数组$fa$存储的值，不断递归访问元素的父节点，直到到达树根节点。如果两个元素的树根节点一样，则说明它们属于同一个集合；如果两个元素的树根节点不一样，则说明它们不属于同一个集合。

举个例子

![基于森林实现并查集](../../image/基于森林实现并查集.png)

实现代码如下：

In [ ]:
class UnionFind:
    def __init__(self, n) -> None:
        self.fa = [i for i in range(n)]
    
    def find(self, x):
        if x == self.fa[x]:
            return x
        self.fa[x] = self.find(self.fa[x])
        return self.fa[x] # 返回根节点
    
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x == root_y:
            return False
        self.fa[root_x] = root_y
        return True
    
    def is_connected(self, x, y):
        return self.find(x) == self.find(y)

# 路径压缩

在集合很大或者树很不平衡时，使用上述「快速合并」思路实现并查集的代码效率很差，最坏情况下，树会退化成一条链，单次查询的时间复杂度高达$O(n)$。并查集的最坏情况如下图所示。为了避免出现最坏情况，一个常见的优化方式是「路径压缩」。

![路径压缩](../../image/路径压缩.png)

### 隔代压缩

> **隔代压缩**：在查询时，两步一压缩，一直循环执行「把当前节点指向它的父亲节点的父亲节点」这样的操作，从而减小树的深度。

![隔代压缩](../../image/隔代压缩.png)

In [ ]:
def find(self, x):
    while x != self.fa[x]:
        self.fa[x] = self.fa[self.fa[x]]
        x = self.fa[x]
    return x

### 完全压缩

> **完全压缩**：在查询时，把被查询的节点到根节点的路径上的所有节点的父节点设置为根节点，从而减小树的深度。也就是说，在向上查询的同时，把在路径上的每个节点都直接连接到根上，以后查询时就能直接查询到根节点。

![完全压缩](../../image/完全压缩.png)

In [ ]:
def find(self, x):
    if x != self.fa[x]:
        self.fa[x] = self.find(self.fa[x])
    return self.fa[x]

# 按秩合并

### 按深度合并

> **按深度合并（Unoin By Rank）**：在每次合并操作时，都把「深度」较小的树根节点指向「深度」较大的树根节点。

我们用一个数组$rank$记录每个根节点对应的树的深度（如果不是根节点，其$rank$值相当于以它作为根节点的子树的深度）。

初始化时，将所有元素的$rank$值设为1。在合并操作时，比较两个根节点，把$rank$值较小的根节点指向$rank$值较大的根节点上合并。

![按深度合并](../../image/按深度合并.png)

In [ ]:
class UnionFind:
    def __init__(self, n):                          # 初始化
        self.fa = [i for i in range(n)]             # 每个元素的集合编号初始化为数组 fa 的下标索引
        self.rank = [1 for i in range(n)]           # 每个元素的深度初始化为 1
    
    def find(self, x):                              # 查找元素根节点的集合编号内部实现方法
        while self.fa[x] != x:                      # 递归查找元素的父节点，直到根节点
            self.fa[x] = self.fa[self.fa[x]]        # 隔代压缩
            x = self.fa[x]
        return x                                    # 返回元素根节点的集合编号

    def union(self, x, y):                          # 合并操作：令其中一个集合的树根节点指向另一个集合的树根节点
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x == root_y:                        # x 和 y 的根节点集合编号相同，说明 x 和 y 已经同属于一个集合
            return False
        
        if self.rank[root_x] < self.rank[root_y]:   # x 的根节点对应的树的深度 小于 y 的根节点对应的树的深度
            self.fa[root_x] = root_y                # x 的根节点连接到 y 的根节点上，成为 y 的根节点的子节点
        elif self.rank[root_y] > self.rank[root_y]: # x 的根节点对应的树的深度 大于 y 的根节点对应的树的深度
            self.fa[root_y] = root_x                # y 的根节点连接到 x 的根节点上，成为 x 的根节点的子节点
        else:                                       # x 的根节点对应的树的深度 等于 y 的根节点对应的树的深度
            self.fa[root_x] = root_y                # 向任意一方合并即可
            rank[y] += 1                            # 因为层数相同，被合并的树必然层数会 +1
        return True

    def is_connected(self, x, y):                   # 查询操作：判断 x 和 y 是否同属于一个集合
        return self.find(x) == self.find(y)


### 按大小合并

> **按大小合并（Unoin By Size）**：这里的大小指的是集合节点个数。在每次合并操作时，都把「集合节点个数」较少的树根节点指向「集合节点个数」较大的树根节点。

我们用一个数组$size$记录每个根节点对应的集合节点个数（如果不是根节点，其$size$值相当于以它作为根节点的子树的集合节点个数）。

初始化时，将所有元素的$size$值设为1。在合并操作时，比较两个根节点，把$size$值较小的根节点指向$size$值较大的根节点上合并。

![按大小合并](../../image/按大小合并.png)

In [ ]:
class UnionFind:
    def __init__(self, n):                          # 初始化
        self.fa = [i for i in range(n)]             # 每个元素的集合编号初始化为数组 fa 的下标索引
        self.size = [1 for i in range(n)]           # 每个元素的集合个数初始化为 1
    
    def find(self, x):                              # 查找元素根节点的集合编号内部实现方法
        while self.fa[x] != x:                      # 递归查找元素的父节点，直到根节点
            self.fa[x] = self.fa[self.fa[x]]        # 隔代压缩优化
            x = self.fa[x]
        return x                                    # 返回元素根节点的集合编号

    def union(self, x, y):                          # 合并操作：令其中一个集合的树根节点指向另一个集合的树根节点
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x == root_y:                        # x 和 y 的根节点集合编号相同，说明 x 和 y 已经同属于一个集合
            return False
        
        if self.size[root_x] < self.size[root_y]:   # x 对应的集合元素个数 小于 y 对应的集合元素个数
            self.fa[root_x] = root_y                # x 的根节点连接到 y 的根节点上，成为 y 的根节点的子节点
            self.size[root_y] += self.size[root_x]  # y 的根节点对应的集合元素个数 累加上 x 的根节点对应的集合元素个数
        elif self.size[root_x] > self.size[root_y]: # x 对应的集合元素个数 大于 y 对应的集合元素个数
            self.fa[root_y] = root_x                # y 的根节点连接到 x 的根节点上，成为 x 的根节点的子节点
            self.size[root_x] += self.size[root_y]  # x 的根节点对应的集合元素个数 累加上 y 的根节点对应的集合元素个数
        else:                                       # x 对应的集合元素个数 小于 y 对应的集合元素个数
            self.fa[root_x] = root_y                # 向任意一方合并即可
            self.size[root_y] += self.size[root_x]
            
        return True

    def is_connected(self, x, y):                   # 查询操作：判断 x 和 y 是否同属于一个集合
        return self.find(x) == self.find(y)


# 算法分析

首先我们来分析一下并查集的空间复杂度。在代码中，我们主要使用了数组$fa$来存储集合中的元素。如果使用了「按秩合并」的优化方式，还会使用数组$rank$或者数组$size$来存放权值。因为空间复杂度取决于元素个数，不难得出空间复杂度为$O(n)$。

在同时使用了「路径压缩」和「按秩合并」的情况下，并查集的合并操作和查找操作的时间复杂度可以接近于$O(1)$。最坏情况下的时间复杂度是$O(m×α(n))$。这里的$m$是合并操作和查找操作的次数，$\alpha(n)$是Ackerman函数的某个反函数，其增长极其缓慢，也就是说其单次操作的平均运行时间可以认为是一个很小的常数。

总结一下：

- 并查集的空间复杂度：$O(n)$。
- 并查集的时间复杂度：$O(m×α(n))$。

# Tips

使用并查集的策略（仅供参考）是这样：使用<font color='red'>「隔代压缩」</font>，一般不使用<font color='blue'>「按秩合并」</font>。

这样选择的原因是既能保证代码简单易写，又能得到不错的性能。如果这样写的性能还不够好的话，再考虑使用<font color='blue'>「按秩合并」</font>。